In [1]:
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
with open('questions', 'r', encoding='utf-8') as file:
    response_text = file.read()

soup = BeautifulSoup(response_text, 'html.parser')
questions_tag = soup.find('questions')

questions_data = questions_tag['v-bind:questions']
questions_type = {v: k for k, v in json.loads(questions_tag['v-bind:question-types']).items()}

question_image_base_url = questions_tag['v-bind:question_image_path'][1:-1]
answer_image_base_url = questions_tag['v-bind:image_path'][1:-1]

questions_list = json.loads(questions_data)

In [3]:
questions_df = pd.DataFrame(questions_list)
questions_df.drop(columns=['is_last_question','duration', 'number'], inplace=True)
questions_df['type'] = questions_df['type'].replace(questions_type).str[14:]
questions_df.sort_values(by='type', inplace=True)

def normalize_answers(answers):
    if isinstance(answers, list):
        answers_sorted = sorted(answers, key=lambda x: x['uid'])
        return json.dumps(answers_sorted, ensure_ascii=False)
    return json.dumps(answers, ensure_ascii=False)

questions_df['normalized_answers'] = questions_df['answers'].apply(normalize_answers)
questions_df.drop_duplicates(subset=['text', 'normalized_answers'], inplace=True)
questions_df.drop(columns=['normalized_answers'], inplace=True)
questions_df

,type,text,text_after,images,answers,answers_draggable
5,MANY_FROM_MANY,К простым плюс-РНК-содержащим вирусам семейств...,,[],"[{'uid': 29918, 'answer': 'вирус иммунодефицит...",[]
17,MANY_FROM_MANY,Выберите возможные механизмы участия специфиче...,,[],"[{'uid': 9066, 'answer': 'антитело-зависимая к...",[]
24,MANY_FROM_MANY,Обнаружение вирусов в однослойных культурах кл...,,[],"[{'uid': 14651, 'answer': 'в реакции гемадсорб...",[]
13,MANY_FROM_MANY,Культивирование вирусов в условиях лаборатории...,,[],"[{'uid': 6861, 'answer': 'куриных эмбрионов', ...",[]
26,MATCHING,Установите соответствие между типом профилакти...,,[],"[{'uid': 20156, 'answer': 'живая вакцина', 'im...","[{'uid': 0, 'answer': 'профилактика герпеса', ..."
4,MATCHING,Установите соответствие между вирусом и морфол...,,[],"[{'uid': 11388, 'answer': 'ротавирус', 'images...","[{'uid': 0, 'answer': 'сложный (оболочечный), ..."
9,MATCHING,Установите соответствие между вирусом и таксон...,,[],"[{'uid': 12384, 'answer': 'вирус SARS-CoV-2', ...","[{'uid': 0, 'answer': 'Flaviviridae', 'images'..."
28,MATCHING,﻿Установите соответствие между группой препара...,,[],"[{'uid': 5374, 'answer': 'аномальные нуклеозид...","[{'uid': 0, 'answer': 'нарушение нарезания вир..."
15,MATCHING,Установите соответствие между инфекционным аге...,,[],"[{'uid': 27664, 'answer': 'простой вирус', 'im...","[{'uid': 0, 'answer': 'инфекционная РНК', 'ima..."
29,ONE_FROM_MANY,Какая из перечисленных ферментативных активнос...,,[6659a7e0eaa9a1.48143601.png],"[{'uid': 29188, 'answer': 'ДНК-лигаза', 'image...",[]


In [7]:
def render_questions_to_text(df):
    lines = []
    for _, row in df.iterrows():
        question_type = row['type']
        if row['images']:
            lines.append(f"{question_type} {row['text']}")
            for i, image in enumerate(row['images'], start=1):
                lines.append(f"КАРТИНКА ВОПРОСА {i}: {question_image_base_url}{image}")
        else:
            lines.append(f"{question_type} {row['text']}")
        
        if question_type == 'MATCHING':  # Handle "MATCHING" type questions
            for answer in row['answers']:
                answer_text = answer['answer']
                image_ids = ', '.join(answer['images']) if answer['images'] else ''
                lines.append(f"{answer_text} ({answer_image_base_url}{image_ids})" if image_ids else answer_text)
            
            max_answer_length = max(
                max((len(x['answer']) for x in row['answers']), default=0),
                max((len(x['answer']) for x in row['answers_draggable']), default=0)
            )
            lines.append('-' * max_answer_length)

            for answer in row['answers_draggable']:
                answer_text = answer['answer']
                image_ids = ', '.join(answer['images']) if answer['images'] else ''
                lines.append(f"{answer_text} ({answer_image_base_url}{image_ids})" if image_ids else answer_text)
        else:
            for answer in row['answers']:
                answer_text = answer['answer']
                image_ids = ', '.join(answer['images']) if answer['images'] else ''
                lines.append(f"{answer_text} ({answer_image_base_url}{image_ids})" if image_ids else answer_text)
        
        lines.append('')  # Add an empty line between questions
    return '\n'.join(lines)

questions_text = render_questions_to_text(questions_df)

output_file_path = str(datetime.datetime.now())
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(questions_text)

OSError: [Errno 22] Invalid argument: '2024-06-09 23:30:50.084254'